In [1]:
import warnings

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from src.data.dataloader import MURADataset
from src.models.models import CNN
from netcal.metrics import ECE
from src.models.utils import pred, get_variable
from torch.autograd import Variable
from torch.nn import functional as F

import cv2

import numpy as np

In [3]:
# simple implementation of CAM in PyTorch for the networks such as ResNet, DenseNet, SqueezeNet, Inception
# last update by BZ, June 30, 2021

import io
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
import cv2
import json

# input image
#LABELS_file = 'imagenet-simple-labels.json'
image_file = 'data/MURA-v1.1/valid/XR_SHOULDER/patient11723/study1_positive/image3.png'

model_path = 'models/STATEtrained_model_epocs2_24-03-2022_14.pt'

device = "cuda" if torch.cuda.is_available() else "cpu"

use_cuda = torch.cuda.is_available()


net = CNN(input_channels=3, input_height=256, input_width=256, num_classes=7).to(
    device
)

net.eval()

net.load_state_dict(
    torch.load(
        model_path,
        map_location=torch.device(device),
    )
)

net.eval()

# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

net._modules.get('conv3').register_forward_hook(hook_feature)

# get the softmax weight
params = list(net.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        print(weight_softmax[idx].shape)
        cam = weight_softmax.dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam



preprocess = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

# load test image
img_pil = Image.open(image_file).convert("RGB")
img_tensor = preprocess(img_pil)
img_variable = Variable(img_tensor.unsqueeze(0))
logit = net(img_variable)

# load the imagenet category list
classes = [0,1,2,3,4,5,6]


h_x = F.softmax(logit, dim=1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()
idx = idx.numpy()

# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

# generate class activation mapping for the top1 prediction
CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

# render the CAM and output
print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
img = cv2.imread(image_file)
height, width, _ = img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.3 + img * 0.5
cv2.imwrite('reports/CAM.jpg', result)



0.989 -> 0
0.011 -> 4
0.000 -> 1
0.000 -> 3
0.000 -> 2
()
output CAM.jpg for the top1 prediction: 0


True

In [ ]:
STOP

In [ ]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    print(weight_softmax.shape)
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax.dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [ ]:
# Model
model_path = 'models/STATEtrained_model_epocs2_24-03-2022_14.pt'

# Inputs
batch_size = 1
shuffle = True
num_workers = 1

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

use_cuda = torch.cuda.is_available()

# Transforming the data, such that they all follow the same path
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

test_set = MURADataset(
    "data",
    "MURA-v1.1/valid_labeled_studies.csv",
    "MURA-v1.1/valid_image_paths.csv",
    transform=transform,
)

test_loader = DataLoader(
    dataset=test_set,
    shuffle=False,  # very important!
    batch_size=batch_size,
    num_workers=num_workers,
)

model = CNN(input_channels=3, input_height=256, input_width=256, num_classes=7).to(
    device
)

model.eval()

model.load_state_dict(
    torch.load(
        model_path,
        map_location=torch.device(device),
    )
)

In [ ]:
params = list(model.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())


In [ ]:
features_blobs = []

def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

model._modules.get('conv3').register_forward_hook(hook_feature)

In [ ]:
inputs, labels = next(iter(test_loader))

inputs, labels = Variable(get_variable(inputs)), Variable(
                get_variable(labels)
            )

In [ ]:
logit = model(inputs)
h_x = F.softmax(logit, dim=1)
probs, idx = h_x.sort(0, True)

# generate class activation mapping for the top1 prediction
CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

In [ ]:
# render the CAM and output
print('output CAM.jpg for the top1 prediction: %s'%labels[idx[0]])
img = cv2.imread('data/MURA-v1.1/valid/XR_WRIST/patient11185/study1_positive/image1.png')
height, width, _ = img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.3 + img * 0.5
cv2.imwrite('CAM.jpg', result)

In [ ]:
CAMs[0].shape

In [ ]:
weight_softmax[1]

In [ ]:
test_set[0]

In [ ]:
test_loader.item

In [ ]:
inputs.shape

In [ ]:
test_set